## Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read csv

In [ ]:
file = "Pure/pure_pids.csv"
df = pd.read_csv(file, usecols=['objid', 'bands', 'pid1', 'pid2'], dtype=object)
total = len(df)
print("Total objects = {}".format(total))

## Band-Count and Object-Bands Dicts

In [ ]:
band_stat = {b:[] for b in "ugri"}
objid_stat = {}

for i in range(total) :
    objid = df.loc[i, 'objid']
    bands = df.loc[i, 'bands']
    for b in bands :
        band_stat[b].append(df.loc[i, 'objid'])
        
    objid_stat[objid] = bands

## Band Correlation

In [ ]:
corrs = {}
for b1 in "ugri" :
    corr = {b:0 for b in "ugri" if b != b1}

    other_count = []
    single_band = 0
    for objid in band_stat[b1] :
        other = 0
        for b2 in objid_stat[objid] :
            if b1 != b2 :
                other += 1
                corr[b2] += 1

        if not other :
            single_band += 1
        else :
            other_count.append(other)

    uq, cnts = np.unique(np.array(other_count), return_counts=True)
    other_count = {uq[i]:cnts[i] for i in range(len(uq))}
    corrs[b1] = (corr, single_band, other_count)

# Overall Band Statistics

In [ ]:
max_corr = max([
    max([u for u,_ in corrs[b][2].items()])
])
print ("Maximum simultaneous detection of any object is in {} bands".format(max_corr+1))

max_band, max_count = '', 0
for b in "ugri" :
    if len(band_stat[b]) > max_count :
        max_count = len(band_stat[b])
        max_band = b
print ("Maximum detections occured in {}-band with {} detections".format(max_band, max_count))

## Individual Band Statitics

In [ ]:
for band in "ugri" :
    print ("INFO FOR {}-BAND".format(band))
    print ("Total detections in {}-band = {}".format(band, len(band_stat[band])))
    print ("Exclusive detections in {}-band".format(band, corrs[band][1]))
    print ("Band correlation with {}-band - ".format(band))
    for b, c in corrs[band][0].items() :
        print ("\t{}-band : {}".format(b, c))

    print ("Correlation count with {}-band - ".format(band))
    for u, c in corrs[b][2].items() :
        print ("\tNumber of {} other simultaneous detection(s) : {}".format(u, c))

    corr_max, corr_cnt = '', 0
    for b, c in corrs[band][0].items() :
        if c > corr_cnt :
            corr_max = b
            corr_cnt = c
    print ("Max correlation of {}-band is with {}-band carrying {} detections".format(band, corr_max, corr_cnt))
    
    if band != 'i' :
        print (65*"-")